# Uczenie maszynowe - lab 6 - Enkodery

Monika Etrych

## 1. AutoEnkoder (AE)

Zadanie 1.1 - Dlaczego sigmoid jest odpowiednią funkcją aktywacji w ostatniej warstwie dekodera w tym przypadku? (0.25pkt)

Funkcja sigmoid zwraca wartość z przedziału [0, 1].  W naszym przypadku używamy sigmoid aby wyjściowe obrazy były zakodowane tak jak wejściowe - tzn. wartości ciągłe.


Zaimplementuj funkcję do wizualizacji reprezentacji obrazów ze zbioru testowego w ukrytej przestrzeni 2-wymiarowej. Wyświetl wizualizację.

In [ ]:
def plot_latent_space(model, data):
    encoded = model.encoder(data)
    
    data_reshaped = encoded.reshape(data.shape[0], -1)
    
    pca = PCA(n_components=2)
    latent_2d = pca.fit_transform(data_reshaped)
    
    plt.figure(figsize=(8, 6))
    plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c='green', alpha=0.8)
    plt.title('Latent Space Plot - AE')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.grid(True)
    plt.show()

![img1](./img/1.PNG)

Zadanie 1.3. Wybierz ze zbioru testowego dwa obrazy z różnymi liczbami. Dobierz takie liczby, dla których spodziewasz się, że odkodowanie średniej z ich zenkodowanych reprezentacji będzie miało sens. Wybierz dwie takie pary.  
Dla każdej z par:  
* Wyświetl wybrane liczby.  
* Użyj enkodera do uzyskania 2-wymiarowych reprezentacji każdej liczby.  
* Wylicz średnią z tych reprezentacji.  
* Użyj dekodera na uzyskanej średniej.  
* Wyświetl wynik.
* Skomentuj wynik - czy przypomina jakąś liczbę? Czy takiego wyniku się spodziewałaś/eś?  

(0.25pkt)


Łączna liczba punktów do uzyskania za tę część: 0.75

In [ ]:
images = [x_test[0], x_test[2], x_test[4], x_test[9]]

n = len(images)
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(images[i], cmap='Greys_r')
    if i < 2: plt.title("pair 1") 
    else: plt.title("pair 2")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

![img2](./img/2.PNG)

In [ ]:
seven = autoencoder.encoder.predict(np.expand_dims(images[0], axis=0))
two = autoencoder.encoder.predict(np.expand_dims(images[1], axis=0))
four = autoencoder.encoder.predict(np.expand_dims(images[2], axis=0))
nine = autoencoder.encoder.predict(np.expand_dims(images[3], axis=0))

seven_two_mean = np.mean(np.array([seven, two]), axis=0 )
four_nine_mean = np.mean(np.array([four, nine]), axis=0 )

seven_two_dec = autoencoder.decoder.predict(seven_two_mean)
four_nine_dec = autoencoder.decoder.predict(four_nine_mean)

In [ ]:
images_dec = [np.squeeze(seven_two_dec), np.squeeze(four_nine_dec)]

n = len(images_dec)
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(images_dec[i], cmap='Greys_r')
    if i < 1: plt.title("pair 1") 
    else: plt.title("pair 2")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

![img3](./img/3.PNG)

## 2. AutoEnkoder wariacyjny (VAE) 

Zadanie 2.1. Dlaczego powyższa implementacje CVAE nie stosuje żadnej aktywacji w ostatniej warstwie enkodera? Czy jakaś funkcja by się tutaj nadawała? (0.25pkt)

Ponieważ model uczy się generować nowe obrazy na podstawie rozkładu prawdopodobieństwa obliczanego na podstawie binarnej reprezentacji, to wyjściowe dane niekoniecznie muszą być z zakresu [0, 1]. Nie ustawianie funkcji aktywacji w wartstwie wyjściowej daje możliwość uzyskania bardziej zróżnicowanych wyników przez co model ma większe możliwości generatywne. Później i tak dokonuje się zaokrąglenia aby uzyskać ponownie reprezemtację binarną (jeśli to jest naszym celem).


Narysujmy teraz, podobnie jak wcześniej dla AE, dwuwymiarową przestrzeń ukrytą. Zaimplementuj funkcję plot_latent_space, która zenkoduje zbiór danych, a następnie wyświetli każdy punkt wraz z odchyleniem standardowym.

In [ ]:
def plot_latent_space(model, data):
    mean, log_var = model.encode(data)
    
    std_dev = tf.exp(0.5 * log_var)
    
    epsilon = tf.random.normal(tf.shape(std_dev))
    
    # normalization
    z = mean + epsilon * std_dev 
    z = z.numpy()
    
    plt.figure(figsize=(8, 6))
    plt.scatter(z[:, 0], z[:, 1], c='blue', alpha=0.8)
    plt.title('Latent Space Plot - CVAE')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.grid(True)
    plt.show()

![img4](./img/4.png)

#### Zadanie 2.2. Skomentuj wynik uzyskany przy użyciu funkcji plot_latent_images. Zwróć uwagę na jakość/sensowność rysowanych liczb. Porównaj wykres do analogicznego wykresu dla modelu AE. Zamieść w raporcie wykresy. (0.25pkt)  
### Autoenkoder
![5](./img/5.png)
### CVAE
![6](./img/6.png)

Wyniki uzyskane przez CVAE są mniej czytelne, trudniej domyśleć się jaką cyfrę wygenerował model.


  
#### Zadanie 2.3. Porównaj wyniki funkcji _plot_latent_space_ dla AE oraz VAE. Zwróć uwagę na "gęstość" punktów oraz zakres wartości. Zamieść w raporcie wykresy. (0.25pkt)   
  
W AE zakres wartości jest szerszy. W VAE punkty są gęściej rozmieszczone.

#### Zadanie 2.4. Dla tych samych par obrazów, na których pracowałaś/eś w ostatnim zadaniu dot. AE, przygotuj reprezentacje ukryte z pomocą wytrenowanego VAE i odkoduj średnie z reprezentacji. Skomentuj wyniki, porównaj z wynikami z AE. (0.25pkt)  

![7](./img/7.png)

Wyniki wyszły bardzo podobne do AE, a to dlatego, że enkoder w obu przypadkach jest bardzo podobny do siebie, różnią się jedynie ostatnią warstwą. AE zwraca wektor o długości latent_dim, a VAE dwuktotny taki wektor.

#### Zadanie 2.5. Wróć do funkcji _compute_loss_. Człony _logpz_ oraz _logqz\_x_ związane są z obliczaniem KL-divergence pomiędzy $Q(z|X)$ oraz $P(z)$. Zakładamy, że oba te rozkłady są gaussowskie, stąd możemy wykorzystać wzór na KL-divergence dla dwóch rozkładów gaussowskich. Znajdź ten wzór oraz przepisz funkcję _compute_loss_ z jego wykorzystaniem. Zamieść w raporcie przygotowaną formułę. Wytrenuj model ponownie, porównaj wyniki z poprzednią implementacją _compute_loss_. (0.25pkt)

Wychodząc od wzoru na KL dla rozkładów ciągłych:

![formula](./img/f1.png)

Podstawiajać za rozkłady funkcje gęstości:

![formula](./img/f2.png)

Otrzymujemy:

![formula](./img/8.png)

Przy dodatkowym założeniu, że rozkład P(X) jest normalny zestandaryzowany mean1=0, var1=1, uproszczony wzór to:

![formula](./img/f3.png)

Dodatkowo wzór na loss:

![formula](./img/9.png)


In [ ]:
def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar) # draw examples
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x) 
  
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3]) # error
  kl_div = -0.5 * tf.reduce_sum(1 + logvar - tf.exp(logvar) - tf.square(mean), axis=-1)
  
  return -tf.reduce_mean(logpx_z - kl_div)

### Wyniki

Oryginala implementacja:

![formula](./img/r1.png)

Zmieniona implementacja:

![formula](./img/r2.png)

Wyniki wychodzą takie same.

## 3. Conditioned CVAE
Uzupełnij klasę Cond_CVAE na podstawie klasy CVAE. W tym celu:  

Uzupełnij funkcję, która z każdego obrazu ze zbioru danych x tworzy 9 obrazów z cyfrą na każdej z 9 pozycji siatki 3x3, a także tworzy etykiety y w postaci wektora [cyfra-one-hot, pozycja_x, pozycja_y]. Dla każdej pary z oryginalnego zbioru danych (obraz, etykieta) wylosuj num_imgs par, które znajdą się w docelowym zbiorze danych (nie zapisujemy wszystkich 9 możliwości ze względu na ograniczenia RAM).

In [ ]:
import random

def conditioned_mnist(x, y, num_imgs=2):
  x_res = np.empty(shape=(num_imgs*len(x), 42, 42), dtype='float32')  # pusta macierz z wynikami - obrazy x
  y_res = np.empty(shape=(num_imgs*len(y), 12), dtype='float32')  # pusta macierz z wynikami - wektor y: etykieta (10 liczb), pozycja x, pozycja y
  empty_res = np.zeros(shape=(42, 42))  # obraz wynikowy w docelowym rozmiarze, wypełniony zerami

  for el, (arr, label) in enumerate(zip(x, y)):
    to_sample_x = np.empty((9, x.shape[1]*3, x.shape[2]*3), dtype='float32')  # macierz przechowująca 9 wersji obrazu
    to_sample_y = np.empty((9, 12), dtype='float32')  # macierz przechowująca 9 wersji etykiet
    for i in range(3):
      for j in range(3):
        curr_x = empty_res.copy()
        curr_x[i*x.shape[1]: (i+1)*x.shape[1], j*x.shape[2]: (j+1)*x.shape[2]] = arr.reshape((x.shape[1], x.shape[2]))
        curr_y = [*label, i/2, j/2]  # normalizacja
        to_sample_x[3*i+j] = curr_x
        to_sample_y[3*i+j] = curr_y
    idxs = [random.randint(0, 8) for i in range(num_imgs)]  # wylosuj num_imgs indeksów z zakresu [0; 8] jako wektor numpy
    x_res[el*num_imgs: (el+1)*num_imgs] = [to_sample_x[i] for i in idxs]
    y_res[el*num_imgs: (el+1)*num_imgs] = [to_sample_y[i] for i in idxs]
  x_res = x_res.reshape((-1, x.shape[1]*3, x.shape[2]*3, 1))
  return x_res, y_res

1. Uzupełnij funkcję prepare_encoder. Będziemy mieć dwa wejścia do modelu - jedno na obraz, jedno na wektor cech [etykieta, pos_x, pos_y]. Przeprocesuj obraz z pomocą warstw konwolucyjnych (możesz wykorzystać implementację z CVAE). Możesz też przygotować kilka warstw, które zajmą się wektorem cech. Użyj warstwy konkatenującej wyniki z przetwarzania obrazu i wektora cech. Za tą warstwą znajdzie się warstwa gęsta, wyliczająca średnią i logvar.

2. Uzupełnij funkcję prepare_decoder. Tu również mamy do czynienia z dwoma wejściami - jedno przyjmuje szum, drugie wektor cech. Połącz oba wejścia i przygotuj dekoder. Możesz skorzystać z implementacji CVAE, ale będą potrzebne zmiany związane z innym rozmiarem obrazów.

Pozostałe funkcje są już zaimplementowane. Przyjrzyj się im. Co się zmieniło względem implementacji CVAE?

In [ ]:
def prepare_encoder(self):
    input_img = tf.keras.layers.Input(shape=(42, 42, 1))
    input_cond = tf.keras.layers.Input(shape=(12, ))
    conv_img = tf.keras.layers.Conv2D(
        filters=64, kernel_size=3, strides=(3, 3), activation='relu')(input_img)
    conv_img = tf.keras.layers.Conv2D(
        filters=64, kernel_size=3, strides=(3, 3), activation='relu')(conv_img)
    flatten_img = tf.keras.layers.Flatten()(conv_img)
    
    x = tf.keras.layers.Concatenate()([flatten_img, input_cond])
    # No activation
    x = tf.keras.layers.Dense(latent_dim + latent_dim)(x)
    return  tf.keras.Model([input_img, input_cond], [x])

def prepare_decoder(self):
    input_latent = tf.keras.layers.Input(shape=(latent_dim,))  # noice 
    input_cond = tf.keras.layers.Input(shape=(12, ))  # labels, pos
    inputs = tf.keras.layers.Concatenate()([input_latent, input_cond])
    
    x = tf.keras.layers.Dense(units=7*7*64, activation=tf.nn.relu)(inputs)
    x = tf.keras.layers.Reshape(target_shape=(7, 7, 64))(x)
    x = tf.keras.layers.Conv2DTranspose(
            filters=64, kernel_size=3, strides=2, padding='same',
            activation='relu')(x)
    x = tf.keras.layers.Conv2DTranspose(
            filters=32, kernel_size=3, strides=2, padding='same',
            activation='relu')(x)
    x = tf.keras.layers.Conv2DTranspose(
        filters=32, kernel_size=3, strides=2, padding='same',
        activation='relu')(x)
    
    return  tf.keras.Model([input_latent, input_cond], [x])

W pozostałych funkcjach (a konkretniej w funkcji sample) uwzględniono wektor cech - cond.

### Funkcja kosztu

In [ ]:
def compute_loss(model, x):
  data, cond = x
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z, cond)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=data)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)

Zadanie 3.1. Sprawdź jakość modelu dla 3 różnych wartości latent_dim (trzeba dla każdej z nich osobno wytrenować model). Niech będą od siebie znacząco różne, np. 2, 25, 100. Przy większym latent_dim może być potrzebnych więcej epok.  
Punktacja:  
Skuteczny trening dla jednej wartości latent dim = 1 pkt.  
0.5 pkt za testy dla dwóch innych wartości latent_dim (po 0.25 za każdą). Punkty będą przyznane, jeśli model będzie w stanie generować _sensowne_ wyniki.  
  
Zadanie 3.2. Wykonaj dla najlepszego modelu z punktu 3.1.:
* Wybierz przykład ze zbioru testowego (obraz + etykieta).  
* Przepuść próbkę przez enkoder, uzyskaj reprezentację _z_.  
* Dla każdego z 9 możliwych wektorów [poprawna_etykieta, pos_x, pos_y] przepuść przez dekoder reprezentację _z_ wraz z informacją o etykiecie i położeniu. Wyświetl uzyskany obraz. Skomentuj wyniki - czy za każdym razem uzyskano oczekiwaną liczbę w oczekiwanym miejscu? Jeśli nie, to co może być przyczyną? (0.25 pkt)  

Zadanie 3.3. Powtórz zadanie 3.2, ale tym razem jako reprezentację _z_ wykorzystaj wartości wylosowane z rozkładu normalnego oraz wybierz dowolną etykietę. Skomentuj wyniki - czy za każdym razem uzyskano oczekiwaną liczbę w oczekiwanym miejscu? (0.25 pkt)


Za tę część: 2 punkty

## Conditioned GAN
Zadanie 4.1. 
Wygeneruj po jednym obrazie z każdą liczbą z pomocą generatora. Oceń jakość wyników. Jeśli jakość modelu pozostawia wiele do życzenia, spróbuj go poprawić, np. zwiększając liczbę epok bądź zmieniając definicję generatora/dyskryminatora.  
Punktacja: 1 pkt za poprawnie przeprowadzony trening i wykonane zadanie 4.1.